03_clean_tax_parcels.py

In [5]:
import dask_geopandas
import pandas as pd
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
INPUT_DATA_TAX_PARCELS_EAST = "../../data/01_raw/Statewide_parcels_SHP/L3_TAXPAR_POLY_ASSESS_EAST.shp"
INPUT_DATA_TAX_PARCELS_WEST = "../../data/01_raw/Statewide_parcels_SHP/L3_TAXPAR_POLY_ASSESS_WEST.shp"
OUTPUT_DATA_TAX_PARCELS = "../../data/02_intermediate/tax_parcels.gpkg"

In [6]:
# Submit job to Oscar.
cluster = SLURMCluster(queue='batch',
                       cores=4,
                       memory='2 GB')
cluster.scale(jobs=8)
client =  Client(cluster)

In [8]:
tax_parcel_columns_to_keep = ['LOC_ID', 'geometry']
tax_parcels_gdf = pd.concat([dask_geopandas.read_file(INPUT_DATA_TAX_PARCELS_EAST,
                                                      npartitions=8,
                                                      columns=tax_parcel_columns_to_keep),
                             dask_geopandas.read_file(INPUT_DATA_TAX_PARCELS_WEST,
                                                      npartitions=8,
                                                      columns=tax_parcel_columns_to_keep)], axis=0)
tax_parcels_gdf = tax_parcels_gdf.to_crs("EPSG:4326")
tax_parcels_gdf = tax_parcels_gdf.dissolve(by='LOC_ID')
tax_parcels_gdf.to_file(OUTPUT_DATA_TAX_PARCELS, driver='GPKG', layer='layer')

ERROR 4: ../../data/01_raw/Statewide_parcels_SHP/L3_TAXPAR_POLY_ASSESS_EAST.shp: No such file or directory


DataSourceError: ../../data/01_raw/Statewide_parcels_SHP/L3_TAXPAR_POLY_ASSESS_EAST.shp: No such file or directory